<a href="https://colab.research.google.com/github/DarkNacho/UTAL_MachineLearning2022-2/blob/main/Entrega_02_Scrub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Proyecto Final - Reconocimiento de Actividad Humana (HAR)


In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder

###Entrega 01 - OBTAIN

Incluir todo el código de la Etapa 01 - OBTAIN creada en la semana anterior

In [2]:
df = pd.read_csv('drive/MyDrive/WISDM_at_v2.0_raw.txt', sep = ',', low_memory=False)

###Entrega 02 - SCRUB

Eliminar filas con datos faltantes

*Uno de los pasos iniciales en la limpieza de datos consiste en eliminar aquellas filas que contengan datos faltantes. Estas líneas generalmente no aportan información y pueden complicar la etapa de entrenamiento y validación de muchos modelos predictivos.*


In [3]:
df = df.replace(";", np.nan)
df = df.replace(" ", np.nan)
df.iloc[:,5] = df.iloc[:,5].str.rstrip(';')

Agregar al DataFrame los nombres de las columnas 

*En ocasiones al crear los data frames el sistema asigna nombres genéricos y correlativos a las columnas. Es recomendable renombrar las columnas por nombres significativos para favorecer el análisis posterior de los datos.*

In [4]:
df.columns = ["user", "activity", "time", "x", "y", "z"]

Visualizar las 5 primeras filas del DataFrame (.head())

*Siempre es buena medida visualizar algunas líneas del dataframe para tener una idea concreta del contenido.*


In [5]:
df.head()

,user,activity,time,x,y,z
0,1679,Walking,1370520469606,-0.499688,-0.604451,-0.22602014
1,1679,Walking,1370520469656,-2.178345,0.713491,0.37201694
2,1679,Walking,1370520469706,-2.797763,1.354899,-0.27763826
3,1679,Walking,1370520469756,-2.167961,-1.327716,-0.5549711
4,1679,Walking,1370520469806,-1.734857,0.818559,-0.16554448


Buscar datos nulos y eliminarlos si es que existen

In [6]:
df.dropna(inplace=True)
df.reset_index(inplace=True, drop=True)
df.isna().sum()

user        0
activity    0
time        0
x           0
y           0
z           0
dtype: int64

Visualizar los tipo de datos a los que pertenece cada columna (.info())

*Es necesario tener un detalle de cuantas variables están considerando y el tipo de datos que pertenecen para verificar si es posible utilizarla directamente para el modelo predictivo o si es necesario realizar una transformación del tipo de dato.*


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2980764 entries, 0 to 2980763
Data columns (total 6 columns):
 #   Column    Dtype  
---  ------    -----  
 0   user      int64  
 1   activity  object 
 2   time      object 
 3   x         float64
 4   y         float64
 5   z         object 
dtypes: float64(2), int64(1), object(3)
memory usage: 136.4+ MB


Cambiar el tipo de datos de las variables predictoras a valores reales, para realizar la clasificación 

*Muchas de las variables contenidas en el DataFrame originalmente se encuentran en el tipo de dato 'object'. En ese formato las variables no pueden ser interpretadas como vectores. Recordemos que la mayoría de los modelos de Machine Learning estudiados operan en el espacio vectorial. Por esta razón las variables que formarán parte del modelo predictivo deben ser transformadas a un formato numérico como 'Float' o 'integer'.*

Para este proyecto en particular se pide transformar  al tipo de dato Float, las variables del acelerómetro en el eje $x$, $y$ y $z$



In [8]:
df["z"] = pd.to_numeric(df["z"])

Imprimir los tipos de datos de las variables del DataFrame

*Se recomienda volver a ejecutar el comando info para verificar que efectivamente las variables cambiaron su tipo de dato y que ahora se pueden procesar con los modelos predictivos.*


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2980764 entries, 0 to 2980763
Data columns (total 6 columns):
 #   Column    Dtype  
---  ------    -----  
 0   user      int64  
 1   activity  object 
 2   time      object 
 3   x         float64
 4   y         float64
 5   z         float64
dtypes: float64(3), int64(1), object(2)
memory usage: 136.4+ MB


Eliminar las columnas que no se utilizaran para la clasificación

*En este caso deberíamos conservar la actividad respectiva y las variables asociadas al acelerómetro*


In [10]:
df.drop(columns={"time", "user"}, inplace=True)
df.head()

,activity,x,y,z
0,Walking,-0.499688,-0.604451,-0.226020
1,Walking,-2.178345,0.713491,0.372017
2,Walking,-2.797763,1.354899,-0.277638
3,Walking,-2.167961,-1.327716,-0.554971
4,Walking,-1.734857,0.818559,-0.165544


Utilizar el comando *value_counts* para obtener un detalle de cuantas instancias se tienen por actividad. El objetivo consiste en identificar la presencia del balance o desbalance en las categorías a modelar


In [11]:
df["activity"].value_counts()

Walking      1255922
Sitting       663706
Jogging       438871
Standing      288873
LyingDown     275967
Stairs         57425
Name: activity, dtype: int64

Este proyecto considera datos con un desbalance de clases. Es necesario balancearlos para que los modelos operen de manera adecuada. Existen muchas maneras de realizar un balance, pero en este proyecto utilizaremos una estrategia sencilla e intuitiva. Simplemente debe identificar la actividad que contenga el número menor de instancias. Se debe realizar un muestreo para las otras actividades utilizando ese valor. De esta manera todas las clases tendrán la misma cantidad de instancias. 



In [49]:
def sampling_k_elements(group, k=3):
    if len(group) < k:
        return group
    return group.sample(k)
g = df.groupby('activity')
df = g.apply(sampling_k_elements,g.size().min()).reset_index(drop=True)

Establecer el número de instancias y dimensiones del data frame resultante *(shape)*


In [50]:
df.shape

(344550, 4)

Volver a imprimir la cantidad de instancias por categoría para verificar que todo esté en orden. *(value_counts)*

In [51]:
df["activity"].value_counts()

Jogging      57425
LyingDown    57425
Sitting      57425
Stairs       57425
Standing     57425
Walking      57425
Name: activity, dtype: int64

Volver a visualizar las cinco primeras líneas del DataFrame para revisar que todo esté en orden *(head)*


In [52]:
df.head()

,activity,x,y,z
0,Jogging,3.490000,8.960000,8.049625
1,Jogging,-0.380000,8.080000,1.184970
2,Jogging,-1.850364,-1.323058,2.166748
3,Jogging,-10.920000,10.920000,1.076008
4,Jogging,0.029765,-0.243979,0.137116


Separar las variables predictoras (X) de las ctegorias o etiquetas (y)

In [147]:
data = df.copy()
X = data.drop(columns="activity")
Y = data["activity"]

Estandarizar los datos numericos (*StandardScaler)*

*Para el caso de la estandarización de los datos se pueden usar distintas técnicas, como la normalización 01, estandarización entre -1 y 1 o el denominado Z-Score. En este caso usaremos la función Standard Scaler de Python con sus valores por defecto.*


In [145]:
df_scaled = pd.DataFrame(StandardScaler().fit_transform(X),columns = X.columns)
df_scaled.head()

,x,y,z
0,-0.001704,0.001704,-0.001704
1,-0.001704,0.001704,-0.001704
2,-0.001704,0.001704,-0.001704
3,-0.001704,0.001704,-0.001704
4,-0.001704,0.001704,-0.001704


,x,y,z
0,0.000503,-0.099190,0.337933
1,0.073590,0.020386,0.633446
2,-0.361275,1.588160,-0.103312
3,1.060258,0.437573,-0.844119
4,-0.237028,0.139962,-0.698386


Como último paso debe reconstruir el dataframe pegando las columnas de los variables predicadores (X) y las categorías (y)


In [97]:
union = X.copy()
union["activity"] = Y.copy()
union.head()

,x,y,z,activity
0,3.490000,8.960000,8.049625,Jogging
1,-0.380000,8.080000,1.184970,Jogging
2,-1.850364,-1.323058,2.166748,Jogging
3,-10.920000,10.920000,1.076008,Jogging
4,0.029765,-0.243979,0.137116,Jogging
